### Word Frequency

In [1]:
import string
import pandas as pd
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)

[nltk_data] Downloading package stopwords to /Users/jass/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
postsDF = pd.read_csv('./scrappy/scrappy_do.csv')

def wordCount(post):
    d = {}
    l = str(post).split(" ")
    for w in l:
        w = w.translate(translator) # remove punctuation
        w = w.lower()
        if (d.get(w)):
            d[w] = d[w] + 1
        else:
            d[w] = 1
    return d

def stopUnique(post):
    wc = wordCount(post)
    unique = list(wc.keys())
    filtered = [w for w in unique if w not in stop_words]
    return filtered

postsDF['words'] = postsDF['message'].map(lambda x: stopUnique(x))
postsDF.head()

,date,userid,message,words
0,2002-03-25T05:54:02+00:00,merc1,I personally think that with a few tweaks the ...,"[personally, think, tweaks, c320, could, also,..."
1,2002-03-25T07:06:29+00:00,fredvh,I am debating a new purchase and these two are...,"[debating, new, purchase, two, top, list, pric..."
2,2002-03-25T17:02:27+00:00,blueguydotcom,"Great handling, RWD, excellent engine and the ...","[great, handling, rwd, excellent, engine, best..."
3,2002-03-25T23:04:37+00:00,hungrywhale,And no manual tranny. That may not matter to y...,"[manual, tranny, may, matter, certainly, many,..."
4,2002-03-26T00:44:13+00:00,riez,One beauty of BMW 3 Series is that there are s...,"[one, beauty, bmw, 3, series, many, models, ch..."


In [3]:
dataset = list(postsDF['words'].head())

In [9]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)

frequent_itemsets


,support,itemsets
0,0.4,(2002)
1,0.4,(acura)
2,0.4,(around)
3,0.4,(awd)
4,0.4,(best)
5,0.4,(better)
6,0.4,(bmw)
7,0.4,(convertible)
8,0.4,(coupe)
9,0.6,(dont)


In [ ]:
from mlxtend.frequent_patterns import association_rules

x = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
x

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

frequent_itemsets


In [35]:
dataset

[['I',
  'personally',
  'think',
  'tweaks',
  'C320',
  'could',
  'also',
  'sit',
  'top',
  'group',
  'Its',
  'still',
  'entrylevel',
  'luxury',
  'sedan',
  'Mercedes',
  'would',
  'lead',
  'believe',
  'The',
  'Sport',
  'needs',
  'suspension',
  'retuned',
  'accept',
  '17',
  'inch',
  'tires',
  'without',
  'harsh',
  'ride',
  'MB',
  'stop',
  'forcing',
  'metal',
  'trim',
  'black',
  'leather',
  'everyone',
  'wants',
  'sport',
  'package',
  'C240s',
  '6speed',
  'wouldnt',
  'hurt',
  'either',
  'offered',
  'though',
  'fixed',
  'first',
  'That',
  'said',
  'favorite',
  'gasp',
  'new',
  'A4',
  '30',
  'easily',
  'best',
  'interior',
  'sub40K',
  'car',
  'combine',
  'stunning',
  'looks',
  'body',
  'rigid',
  'ever',
  'made',
  'personal',
  'single',
  'thing',
  'dont',
  'like',
  'grey',
  'around',
  'bottom',
  'This',
  'doesnt',
  'match',
  'colors',
  'color',
  'anyway',
  'excellent',
  'opinion',
  '40K',
  'upcoming',
  'Conv